#Environment for PIP  🛺 🚀 🦾


In this notebook there's the environment for our PIP 🐧 📚

🔖 🦤 *Code based on the standard pybullet [CartPole environment](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/cartpole_bullet.py)*


---

---

* 🎮  **Actions space**: Continous action space, an array of two element indicating the force/acceleration to be applied on x,y (sign and abs value).

* 🎮 **Observations space**: 3 LVDT measures, that is the 3 leg position in the form (x,y,z), and the two angle θ, which is the angle arond z.

  1. ❕  **FORMAT**
  I'm ignoring z coordinate and the tilt angle Ψ, cause the cart must stay on the plane, not move in vertical, nore tilt with respect to the plane.
  **Obeservations** : (x,y,z,yaw) for all legs

  2. ❗ **CONSTRAINTS**:
  Leg next position with rispect to the previous one should not differ for more than 2 mm, that is:
      *distance < 0.002 m*

Just to be sure,let's just put a constrain also to the angle around z: difference between the current and previous should not be more than 2mm / rdisco = 0.005 rad.
Not needed because it's the same constraint of the position, but just in case.

🛑 This notebook needs the gitPython packege to be installed, otherwise it wont work

In [ ]:
!pip install GitPython
!pip install pybullet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 12.5 MB/s eta 0:00:00


In [ ]:
import os, inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
os.sys.path.insert(0, parentdir)

#useful libraries
import logging
import math
import random
import numpy as np
import time
import subprocess

#reinforcement
import gym
from gym import spaces
from gym.utils import seeding

# physics simulator
import pybullet as p
import pybullet_data
from pybullet_utils import bullet_client as bc
from pkg_resources import parse_version

#get git repo
import git

logger = logging.getLogger(__name__)


class RealMegaFufiEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}

  def __init__(self, renders=False):
    # set the render
    self._renders = renders
    self._render_height = 200
    self._render_width = 320
    self._physics_client_id = -1

    #set limits for actions and observations
    self.threshold_distance = 0.002
    self.threshold_yaw = 0.005

    high = np.array(
        [
          np.finfo(np.float32).max, np.finfo(np.float32).max, np.finfo(np.float32).max, self.threshold_yaw * 2, #LVDT 1
          np.finfo(np.float32).max, np.finfo(np.float32).max, np.finfo(np.float32).max, self.threshold_yaw * 2, #LVDT 2
          np.finfo(np.float32).max, np.finfo(np.float32).max, np.finfo(np.float32).max, self.threshold_yaw * 2, #LVDT 3
        ],
        dtype=np.float32,
        )

    self.observation_space = spaces.Box(-high, high, dtype=np.float32)

    action_lim = np.finfo(np.float32).max

    # force may be applied in the 2 direction of xy plane. Should always be 0 in the z direction
    self.action_space = spaces.Box(low=-action_lim, high=action_lim, shape=(2,), dtype=np.float32)

    self.seed()

    # start the bullet physics server and initialize things up
    self.reset()
    self.viewer = None
    self._configure()

  def _configure(self, display=None):
    self.display = display

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  #load model in pybullet already with material defined
  def load_pippa(self):
    # Getting git repo for PIP model
    repo_url = 'https://github.com/Gaianeve/Real_Mega_Fufi.git'
    repo_name = 'Real_Mega_Fufi'

    if not os.path.exists(repo_name):
        git.Repo.clone_from(repo_url, repo_name)
        print(f'Repository {repo_name} cloned successfully.')
    else:
        print(f'Repository {repo_name} already exists.')

    # saving PIP urdf directory for the model
    pippa_path = os.path.join(repo_name, 'Robot', 'PIPPA')
    if os.path.exists(pippa_path):
        os.chdir(pippa_path)
        print(f'Changed directory to {pippa_path}.')
    else:
        print(f'Path {pippa_path} does not exist.')

    ## -------------------------------------------------------------------------------------
    #load model in pybullet
    useFixedBase = False
    flags = p.URDF_INITIALIZE_SAT_FEATURES
    plane_pos = [0,0,-0.1]
    plane = p.loadURDF("plane.urdf", plane_pos, flags = flags, useFixedBase=useFixedBase)

    # position of the base of the model
    PIP_position = [0,0,1.043]
    orientation  = [np.pi,0,0]
    PIP_orientation = p.getQuaternionFromEuler(orientation)

    self.PIPPA_id = p.loadURDF("PIPPA.urdf",basePosition = PIP_position,baseOrientation = PIP_orientation,\
                          useFixedBase=useFixedBase)

    #find link names and indexes
    self.link_name_to_index = {p.getBodyInfo(self.PIPPA_id)[0].decode('UTF-8'):-1,}

    for id in range(p.getNumJoints(self.PIPPA_id)):
      name = p.getJointInfo(self.PIPPA_id, id)[12].decode('UTF-8')
      self.link_name_to_index[name] = id

    ## ------------------------------------------------------------------------------------------------
    ## define material for PIP link

    # AL 6082 (Alluminio)
    lateralFriction_Al = 0.3
    restitution_Al = 0.2
    rollingFriction_Al = 0.001
    spinningFriction_Al = 0.01
    linearDamping_Al = 0.04
    angularDamping_Al = 0.1
    contactStiffness_Al = 3e6
    contactDamping_Al = 800

    # AISI 304 (Acciaio Inox)
    lateralFriction_AISI = 0.4
    restitution_AISI = 0.1
    rollingFriction_AISI = 0.001
    spinningFriction_AISI= 0.05
    linearDamping_AISI = 0.03
    angularDamping_AISI = 0.1
    contactStiffness_AISI = 8e6
    contactDamping_AISI = 1500

    # Acciaio Maraging (Maragià per gli amici)
    lateralFriction_Mar = 0.3
    restitution_Mar = 0.2
    rollingFriction_Mar = 0.001
    spinningFriction_Mar = 0.02
    linearDamping_Mar = 0.04
    angularDamping_Mar = 0.15
    contactStiffness_Mar = 3e6
    contactDamping_Mar = 800

    link_indices_Al = [self.link_name_to_index['Bottom_Link_1'],self.link_name_to_index['Bottom_Link_2'],\
                      self.link_name_to_index['Bottom_Link_3'], self.link_name_to_index['Gamba_Link_1'],\
                      self.link_name_to_index['Gamba_Link_2'], self.link_name_to_index['Gamba_Link_3'],\
                      self.link_name_to_index['Top_core'], self.link_name_to_index['Cover_Link1'], \
                      self.link_name_to_index['Cover_Link_2'], self.link_name_to_index['Cover_Link_3']]

    link_indices_AISI = [self.link_name_to_index['Top_Link_2'], self.link_name_to_index['Top_Link_3'],\
                        self.link_name_to_index['Top_Link_1'], self.link_name_to_index['Colonna_Link_1'],\
                        self.link_name_to_index['Colonna_Link_2'], self.link_name_to_index['Colonna_Link_3']]

    link_indices_Mar = [self.link_name_to_index['Elastic_Joint_Link_1'], self.link_name_to_index['Elastic_Joint_link_2'],\
                        self.link_name_to_index['Elastic_Joint_link_3']]

    # Al 6082
    for link in link_indices_Al:
      p.changeDynamics(self.PIPPA_id, link, lateralFriction=lateralFriction_Al,\
                      restitution=restitution_Al, \
                      spinningFriction= spinningFriction_Al, rollingFriction= rollingFriction_Al,\
                      linearDamping=linearDamping_Al, angularDamping=angularDamping_Al,\
                      contactStiffness = contactStiffness_Al,contactDamping = contactDamping_Al)
    # AISI 304
    for link in link_indices_AISI:
      p.changeDynamics(self.PIPPA_id, link, lateralFriction=lateralFriction_AISI,\
                      restitution=restitution_AISI, \
                      spinningFriction= spinningFriction_AISI, rollingFriction= rollingFriction_AISI,\
                      linearDamping=linearDamping_AISI, angularDamping=angularDamping_AISI,\
                      contactStiffness = contactStiffness_AISI, contactDamping = contactDamping_AISI)
    # Maragià
    for link in link_indices_AISI:
      p.changeDynamics(self.PIPPA_id, link, lateralFriction=lateralFriction_Mar,\
                      restitution=restitution_Mar, \
                      spinningFriction= spinningFriction_Mar, rollingFriction= rollingFriction_Mar,\
                      linearDamping=linearDamping_Mar, angularDamping=angularDamping_Mar,\
                      contactStiffness = contactStiffness_Mar, contactDamping = contactDamping_Mar)

    #get position and yaw of each leg in rest position
    ##--------------------------------------------------------------------------------------------------------------
    leg_1_index = self.link_name_to_index['Gamba_Link_1']
    leg_2_index = self.link_name_to_index['Gamba_Link_2']
    leg_3_index = self.link_name_to_index['Gamba_Link_3']

    position_leg_1_rest, orientation_leg_1_rest = p.getLinkState(self.PIPPA_id, leg_1_index)[0:2]
    position_leg_2_rest, orientation_leg_2_rest = p.getLinkState(self.PIPPA_id, leg_2_index)[0:2]
    position_leg_3_rest, orientation_leg_3_rest = p.getLinkState(self.PIPPA_id, leg_3_index)[0:2]

    yaw_1_rest = p.getEulerFromQuaternion(orientation_leg_1_rest)[2]
    yaw_2_rest = p.getEulerFromQuaternion(orientation_leg_2_rest)[2]
    yaw_3_rest = p.getEulerFromQuaternion(orientation_leg_3_rest)[2]

    #get rest state in the same form of rest state
    state_rest = list(position_leg_1_rest) + [yaw_1_rest] + list(position_leg_2_rest) + \
    [yaw_2_rest] + list(position_leg_3_rest) + [yaw_3_rest]
    self.state_rest = np.array(state_rest)

    return self.PIPPA_id

  def reset(self):
    #    print("-----------reset simulation---------------")
    if self._physics_client_id < 0:
      if self._renders:
        self._p = bc.BulletClient(connection_mode=p.GUI)
      else:
        self._p = bc.BulletClient()
      self._physics_client_id = self._p._client

      p = self._p
      p.resetSimulation()

      ## loading PIPPA in the origin
      self.pippa_id = load_pippa()
      self.timeStep = 0.02
      p.setGravity(0, 0, -9.8)
      p.setTimeStep(self.timeStep)
      p.setRealTimeSimulation(0)

    p = self._p

    ##get PIP in random state applying random force to the bottom bars
     ##--------------------------------------------------------------------------------------------------------

     #get legs index
    leg_1_index = self.link_name_to_index['Gamba_Link_1']
    leg_2_index = self.link_name_to_index['Gamba_Link_2']
    leg_3_index = self.link_name_to_index['Gamba_Link_3']

    #get bottom bars index
    bottom_link_1_index = self.link_name_to_index['Bottom_Link_1']
    bottom_link_2_index = self.link_name_to_index['Bottom_Link_2']
    bottom_link_3_index = self.link_name_to_index['Bottom_Link_3']

    #get bottom bar position to apply forces there
    position_bottom_1, _ = p.getLinkState(self.pippa_id, bottom_link_1_index)[0:2]
    position_bottom_2, _ = p.getLinkState(self.pippa_id, bottom_link_2_index)[0:2]
    position_bottom_3, _ = p.getLinkState(self.pippa_id, bottom_link_3_index)[0:2]

    #apply random force to the bottom bars
    fx, fy = random.uniform(0, 1), random.uniform(0, 1)
    force_module = np.array([fx,fy,0])
    force_position_1 = position_bottom_1
    force_position_2 = position_bottom_2
    force_position_3 = position_bottom_3

    #apply external forces to the bottom edges of the disk
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_1_index,forceObj=force_module, \
        posObj=force_position_1,flags=p.WORLD_FRAME)
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_2_index,forceObj=force_module,\
        posObj=force_position_2,flags=p.WORLD_FRAME)
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_3_index,forceObj=force_module, \
        posObj=force_position_3,flags=p.WORLD_FRAME)

    for _ in range(1000):
        p.stepSimulation()  # Aggiorna la simulazione
        time.sleep(1./240.)  # Ritardo per simulare 240 Hz


    #get legs random positions after random force
    position_leg_1, orientation_leg_1 = p.getLinkState(self.pippa_id, leg_1_index)[0:2]
    position_leg_2, orientation_leg_2 = p.getLinkState(self.pippa_id, leg_2_index)[0:2]
    position_leg_3, orientation_leg_3 = p.getLinkState(self.pippa_id, leg_3_index)[0:2]

    #get euler angles, just yaw needed
    yaw_1 = p.getEulerFromQuaternion(orientation_leg_1)[2]
    yaw_2 = p.getEulerFromQuaternion(orientation_leg_2)[2]
    yaw_3 = p.getEulerFromQuaternion(orientation_leg_3)[2]

    # BUROCRAZIA in order to avoid useless complains
    state_list = list(position_leg_1) + [yaw_1] + list(position_leg_2) + [yaw_2] + list(position_leg_3) + [yaw_3]

    #state array
    self.state = np.array(state_list)
    return np.array(self.state)


  def step(self, action):
    p = self._p

    #comnplains to humans
    err_msg = f"{action!r} ({type(action)}) invalid"
    assert self.action_space.contains(action), err_msg
    assert self.state is not None, "Call reset before using step method."

    #force to be applied
    action_x, action_y = action[0], action[1]
    force_module = np.array([action_x, action_y,0])

    #get bottom bar position to apply forces there
    position_bottom_1, _ = p.getLinkState(self.pippa_id, self.link_name_to_index['Bottom_Link_1'])[0:2]
    position_bottom_2, _ = p.getLinkState(self.pippa_id, self.link_name_to_index['Bottom_Link_2'])[0:2]
    position_bottom_3, _ = p.getLinkState(self.pippa_id, self.link_name_to_index['Bottom_Link_3'])[0:2]

    #apply external forces to the bottom edges of the disk
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_1_index,forceObj=force_module, \
        posObj=position_bottom_1,flags=p.WORLD_FRAME)
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_2_index,forceObj=force_module,\
        posObj=position_bottom_2,flags=p.WORLD_FRAME)
    p.applyExternalForce(
        objectUniqueId=self.pippa_id,linkIndex= bottom_link_3_index,forceObj=force_module, \
        posObj=position_bottom_3,flags=p.WORLD_FRAME)

    #calculate next state
    p.stepSimulation()

    #get state after force
    position_leg_1, orientation_leg_1 = p.getLinkState(self.pippa_id, self.link_name_to_index['Gamba_Link_1'])[0:2]
    position_leg_2, orientation_leg_2 = p.getLinkState(self.pippa_id, self.link_name_to_index['Gamba_Link_2'])[0:2]
    position_leg_3, orientation_leg_3 = p.getLinkState(self.pippa_id, self.link_name_to_index['Gamba_Link_3'])[0:2]

    #get yaw
    yaw_1 = p.getEulerFromQuaternion(orientation_leg_1)[2]
    yaw_2 = p.getEulerFromQuaternion(orientation_leg_2)[2]
    yaw_3 = p.getEulerFromQuaternion(orientation_leg_3)[2]

    # state after force
    next_state_list = list(position_leg_1) + [yaw_1] + list(position_leg_2) + [yaw_2] + list(position_leg_3) + [yaw_3]
    self.state = np.array(next_state_list)

    #calculate linear and angular distance for termination
    distance_linear_1 = np.linalg.norm(self.state[0:3] - self.state_rest[0:3])
    distance_linear_2 = np.linalg.norm(self.state[4:7] - self.state_rest[4:7])
    distance_linear_3 = np.linalg.norm(self.state[8:11] - self.state_rest[8:11])

    distance_angular_1 = np.abs(self.state[3] - self.state_rest[3])
    distance_angular_2 = np.abs(self.state[7] - self.state_rest[7])
    distance_angular_3 = np.abs(self.state[11] - self.state_rest[11])

    #check for dones
    done =  distance_linear_1 > self.threshold_distance \
                or distance_linear_2 > self.threshold_distance \
                or distance_linear_3 > self.threshold_distance \
                or distance_angular_1 > self.threshold_yaw \
                or distance_angular_2 > self.threshold_yaw \
                or distance_angular_3 > self.threshold_yaw
    done = bool(done)

    #assign reward
    reward = 1.0 if not done else 0.0

    #print("state=",self.state)
    return np.array(self.state), reward, done, {}


  def render(self, mode='human', close=False):
    if mode == "human":
      self._renders = True
    if mode != "rgb_array":
      return np.array([])
    base_pos=[0,0,0]
    self._cam_dist = 2.5
    self._cam_pitch = -10.0
    self._cam_yaw = 0
    if (self._physics_client_id>=0):
      view_matrix = self._p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=base_pos,
        distance=self._cam_dist,
        yaw=self._cam_yaw,
        pitch=self._cam_pitch,
        roll=0,
        upAxisIndex=2)
      proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
             aspect=float(self._render_width) /
             self._render_height,
             nearVal=0.1,
             farVal=100.0)
      (_, _, px, _, _) = self._p.getCameraImage(
          width=self._render_width,
          height=self._render_height,
          renderer=self._p.ER_BULLET_HARDWARE_OPENGL,
          viewMatrix=view_matrix,
          projectionMatrix=proj_matrix)
    else:
      px = np.array([[[255,255,255,255]]*self._render_width]*self._render_height, dtype=np.uint8)
    rgb_array = np.array(px, dtype=np.uint8)
    rgb_array = np.reshape(np.array(px), (self._render_height, self._render_width, -1))
    rgb_array = rgb_array[:, :, :3]
    return rgb_array

  def configure(self, args):
    pass

  def close(self):
    if self._physics_client_id >= 0:
      self._p.disconnect()
    self._physics_client_id = -1